In [12]:
from dotenv import load_dotenv
import os

load_dotenv()
TRELLO_API_KEY = os.getenv("TRELLO_API_KEY")
TRELLO_TOKEN = os.getenv("TRELLO_TOKEN")
SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
SLACK_CHANNEL_ID = os.getenv("SLACK_CHANNEL_ID")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
APP_TOKEN = os.getenv("APP_TOKEN")

BOARD_ID = os.getenv("BOARD_ID")

In [13]:
import requests
import json

url = "https://api.trello.com/1/members/me/boards"

headers = {
  "Accept": "application/json"
}

params = {"key": TRELLO_API_KEY, "token": TRELLO_TOKEN}

response = requests.get(url,headers=headers,
   params=params
)


print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))

[
    {
        "closed": false,
        "creationMethod": null,
        "dateClosed": null,
        "dateLastActivity": "2025-05-12T18:45:00.577Z",
        "dateLastView": "2025-05-14T20:16:13.978Z",
        "datePluginDisable": null,
        "desc": "",
        "descData": null,
        "enterpriseOwned": false,
        "id": "681a203d621e973d789852ff",
        "idBoardSource": null,
        "idEnterprise": null,
        "idMemberCreator": "59ee4e53432dced717ece56d",
        "idOrganization": "60c916c058ed4887fd7bc1b5",
        "idTags": [],
        "ixUpdate": "138",
        "labelNames": {
            "black": "",
            "black_dark": "",
            "black_light": "",
            "blue": "",
            "blue_dark": "",
            "blue_light": "",
            "green": "",
            "green_dark": "",
            "green_light": "",
            "lime": "",
            "lime_dark": "",
            "lime_light": "",
            "orange": "",
            "orange_dark": "",
    

# El bueno

In [14]:
def get_trello_boards():
    url = "https://api.trello.com/1/members/me/boards"
    params = {"key": TRELLO_API_KEY, "token": TRELLO_TOKEN}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        boards_dict = {list['name']: list['id'] for list in response.json()}
        return boards_dict
    else:
        print(f"Error: {response.status_code}")
        return None

def get_trello_lists(board_id):
    url = f"https://api.trello.com/1/boards/{board_id}/lists"
    params = {"key": TRELLO_API_KEY, "token": TRELLO_TOKEN}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        lists_dic = {list['name']: list['id'] for list in response.json()}
        return lists_dic
    else:
        print(f"Error: {response.status_code}")
        return None

In [15]:
lists_dict = get_trello_lists(BOARD_ID)
lists_dict

{'Debug': '681a20cf4aa1d815f3e3ba22',
 'Testing': '681a20d148f90456f5c31e69',
 'Production': '681a20e1fa3e0a2ead6ce342',
 'Backlog': '681a20d652b35c64058cfa0c',
 'Done': '681b629f0dd3c843290fec57',
 'In Progress': '6822406ec8fbc4003f22bb77',
 'Pending': '6822409527b29b191a2888f3'}

In [16]:
boards = get_trello_boards()
print("Your Trello Boards:", boards)

Your Trello Boards: {'AgentAgile': '681a203d621e973d789852ff'}


In [17]:
from crewai import LLM
import json

llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.7
)

def parse_user_input(user_message: str) -> tuple:
    prompt = f"""
    Analyze the user message and extract the intent and details:

    Mensaje: {user_message}

    IMPORTANT: Your response MUST be valid JSON with no additional text.
    Answer EXACTLY in this format: {{"intent": "action_type", "details": {{"key": "value"}}}}
    For example: {{"intent": "move_card", "details": {{"card_id": 123, "list": "Done"}}}}
    """
    
    response_text = llm.call(prompt)
    print(f'Respuesta del modelo: {response_text}') #DEBUG
    print(f'Longitud:', len(response_text)) #Debug
    
    # print("Respuesta del modelo:", repr(response_text)) #DEBUG
    
    # Intentar extraer JSON de la respuesta
    try:
        # Buscar el primer { y el último } para extraer solo el JSON
        start = response_text.find('{')
        end = response_text.rfind('}') + 1
        
        if start >= 0 and end > start:
            json_str = response_text[start:end]
            # print("JSON extraído:", json_str)
            response_dict = json.loads(json_str)
            intent = response_dict['intent']
            details = response_dict['details']
            return intent, details
        else:
            print("No se encontró formato JSON en la respuesta")
            return None, None
    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON: {e}")
        return None, None

# Uso


In [18]:
user_message = "Move card in Testing to Backlog"
intent, details = parse_user_input(user_message)
print("Resultado:", intent, details)

Respuesta del modelo: ```json
{"intent": "move_card", "details": {"current_list": "Testing", "destination_list": "Backlog"}}
```
Longitud: 106
Resultado: move_card {'current_list': 'Testing', 'destination_list': 'Backlog'}


In [19]:
def create_trello_card(list_id, name, desc):
    url = "https://api.trello.com/1/cards"

    headers = {
        "Accept": "application/json"}
    
    query ={
        "idList" : list_id,
        "name" : name,
        "desc" : desc,
        'key' : TRELLO_API_KEY,
        'token': TRELLO_TOKEN,
        'pos': 'top'
        # Check docs to see what other params you can add
        #https://developer.atlassian.com/cloud/trello/rest/api-group-cards/#api-cards-post
    }

    response = requests.post(
                url,
                headers=headers,
                params=query
    )
    
    if response.status_code == 200:
        return 'Tarjeta creada con éxito'
    # In future, add a better return, like json response

# create_trello_card(
#     list_id=lists_dict["Debug"],
#     name='test card',
#     desc= 'testing card creation'
#     )


In [20]:
from typing import NewType, Optional

CardId = NewType('CardId', str)
ListId = NewType('ListId', str)
BoardID = NewType('BoardID', str)

In [21]:
def update_trello_card(
        card_id:CardId,
        list_id:ListId,
        name: Optional[str] = None,
        desc: Optional[str] = None,
        pos: Optional[float] = None) -> requests.Response:
    url = f"https://api.trello.com/1/cards/{card_id}"
    
    headers = {
        "Accept": "application/json"}

    query = {
        "key": TRELLO_API_KEY,
        "token": TRELLO_TOKEN,
        "idList": list_id,
        'name' : name,
        'desc' : desc,
        'pos' : pos
    }
    response = requests.put(url,headers=headers, params=query)
    if response.status_code == 200:
        return response
    else:
        return f'Error: {response.status_code}'


# get_cards_in_list

In [22]:
def get_cards_in_list(list_id):
    url = f"https://api.trello.com/1/lists/{list_id}/cards"

    headers = {
      "Accept": "application/json"
    }
    # query = {}
    response = requests.get(url,headers=headers,params=params)

    if response.status_code == 200:
        card_list = {card['name']: card['id'] for card in response.json()}
        return card_list

# Testing

In [25]:
cards_dict = get_cards_in_list(lists_dict['Debug'])
cards_dict

{'test': '68250436d6c3f33606c8d02d',
 'agent-test': '6825043b3666333af353f9d0',
 're-test': '6825043e1c0faf2e3b1f6415'}

In [24]:
update_trello_card(cards_dict['new-card-testing'], lists_dict['Testing'])

KeyError: 'new-card-testing'

In [ ]:
# update_trello_card(card_id=cards_list['second-card'], list_id=lists_dict["Debug"], name='new-card-testing-2', desc='this is a test card', pos=2.0)

In [ ]:
def get_trello_card(id:CardId):
    url = f"https://api.trello.com/1/cards/{id}"
    headers = {
  "Accept": "application/json"
}

    query = {
    'key': TRELLO_API_KEY,
    'token': TRELLO_TOKEN
    }

    response = requests.get(url, headers=headers, params=query)

    if response.status_code == 200:
        card_info = response.json()
        return card_info
    else:
        return f'Error: {response.status_code}'

In [ ]:
get_trello_card(cards_dict['new-card-testing-2'])

In [ ]:
def send_to_slack(message):
    url = "https://slack.com/api/chat.postMessage"
    headers = {"Authorization": f"Bearer {SLACK_BOT_TOKEN}"}
    payload = {"channel": SLACK_CHANNEL_ID, "text": message}
    response = requests.post(url, headers=headers, json=payload)
    
    # Verificar la respuesta completa de Slack
    response_data = response.json()
    if response.status_code == 200 and response_data.get("ok"):
        print(f"Mensaje enviado correctamente: {response_data}")
        return True
    else:
        print(f"Error al enviar mensaje: {response_data}")
        return False

# Llamada a la función (fuera de la definición)
result = send_to_slack("lhkhlkhlkhlk")
print(result)

In [ ]:
from datetime import datetime

def generate_daily_summary(cards):
    # Obtener la fecha actual
    today = datetime.now().date()
    
    summary = "# Daily Stand-Up Summary\n\n"
    summary += f"Date: {today.strftime('%d/%m/%Y')}\n\n"
    
    # Filtrar tarjetas actualizadas hoy (opcional)
    today_cards = []
    for card in cards:
        last_activity = datetime.fromisoformat(card['dateLastActivity'].replace('Z', '+00:00'))
        if last_activity.date() == today:
            today_cards.append(card)
    
    if not today_cards:
        summary += "No cards were updated today.\n"
        return summary
    
    summary += f"## Cards Updated Today ({len(today_cards)})\n\n"
    
    for card in today_cards:
        # Extraer información relevante
        name = card['name']
        description = card['desc'] if card['desc'] else "No description"
        status = "Open" if not card['closed'] else "Closed"
        url = card['url']
        
        # Añadir al resumen
        summary += f"### {name}\n"
        summary += f"- **Status:** {status}\n"
        summary += f"- **Description:** {description}\n"
        summary += f"- **Last Updated:** {card['dateLastActivity']}\n"
        summary += f"- **URL:** [{url}]\n\n"
    
    return summary

In [ ]:
cards_dict

In [ ]:
cards = list(cards_dict.values())
cards

In [ ]:
cards_info = [get_trello_card(card_id)for card_id in cards]


In [ ]:
summary = generate_daily_summary(cards_info)

In [ ]:
send_to_slack(summary)

# Investigar como hacer para que la información que le den al bot lleguen al agente